In [ ]:
import torch
import importlib
import requests
import time
import subprocess
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import llava
import pandas as pd

from llava.model import *
from torch.nn import functional as F
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.eval.run_llava import load_image
from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path
from transformers.generation.streamers import TextIteratorStreamer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from llava.serve.baristia_utils import  load_image_processor
from peft import PeftModel, get_peft_model, PeftConfig

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers import LlamaConfig
from llava.serve.barista import LoraInferenceService
from io import BytesIO
from PIL import Image


from threading import Thread
import matplotlib.pyplot as plt # for making figures

%matplotlib inline
device = "cuda"

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
def compare_weights(net1, net2):
    idx = 0
    for param1, param2 in zip(net1.parameters(), net2.parameters()):
        idx += 1
        if not torch.equal(param1.data, param2.data):
            print("Weights are not equal.")
            return False

    print(f'All {idx} layers are equal.')
    return True

In [ ]:
reference_model_path = "liuhaotian/llava-v1.5-7b"
lora_model_path = '../checkpoints/llava-v1.5-7b-augmented-roastme-lora-13000-4-epochs'
model_path = reference_model_path
load_8bit=False
load_4bit=False
device='cuda'
device_map="auto"
use_flash_attn=False
torch_dtype=torch.bfloat16
kwargs = {
    'torch_dtype': torch_dtype,
    'device_map': device_map,
}

In [ ]:
#Set Path to folder that contains adapter_config.json and the associated .bin files for the Peft model
adapters_name = lora_model_path

# Get PeftConfig from the finetuned Peft Model. This config file contains the path to the base model
peft_config = PeftConfig.from_pretrained(adapters_name)

# W/out this no vision tower?
lora_cfg_pretrained = LlavaConfig.from_pretrained(peft_config.base_model_name_or_path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, use_fast=False)
model = LlavaLlamaForCausalLM.from_pretrained(peft_config.base_model_name_or_path, low_cpu_mem_usage=True, config=lora_cfg_pretrained, **kwargs)

In [ ]:
base_model = LlavaLlamaForCausalLM.from_pretrained(peft_config.base_model_name_or_path, low_cpu_mem_usage=True, **kwargs)

In [ ]:
peft_model_with_lora = PeftModel.from_pretrained(model=model, model_id=lora_model_path, adapter_name="test_lora", torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
compare_weights(model, peft_model_with_lora)

In [ ]:
parameters_1 = [p for p in model.parameters()]
parameters_2 = [p for p in peft_model_with_lora.parameters()]
print(len([p.shape for p in parameters_1]))
len([p.shape for p in parameters_2])

In [ ]:
base_model

In [ ]:
[x for x in base_model.children()]

In [ ]:
[type(layer) for layer in base_model.modules() if "torch.nn.modules.activation" in str(type(layer))]

In [ ]:
[layer for layer in base_model.modules() if isinstance(layer, nn.ReLU) or isinstance(layer, nn.Sigmoid)]

In [ ]:
base_model_parameters = [p for p in base_model.modules()]
base_model_parameters

In [ ]:
activation_functions = (nn.ReLU, nn.Sigmoid, nn.SiLU, nn.GELU)
# activation_layers = [(idx, layer) for idx, layer in enumerate(base_model.modules()) if isinstance(layer, activation_functions)]
activation_layers = [(idx, layer.__class__) for idx, layer in enumerate(base_model.modules())]
print(len([p.shape for p in base_model.parameters()]))
len(activation_layers)

In [ ]:
layer = activation_layers[0]

In [ ]:
[p for p in layer.parameters()]

In [ ]:
for m in base_model.modules():
    # print(m)
    # print(len([p for p in m.parameters()]))
    # print("-----")

    if isinstance(m, activation_functions):
        print(m)
        print([p for p in m.parameters()])

In [ ]:
print(base_model)

In [100]:
df = pd.read_json("../dataset/augmented/full_dataset.json")

KeyError: 'from'

In [106]:
df['title'].apply(len).idxmin()

9119

In [107]:
df.loc[9119]

id                                                 1abgte5_kjnfv4v
title                                                            🤌
image            /home/devonperoutky/LLaVA/dataset/images/1abgt...
comment_id                                                 kjnfv4v
conversations    [{'from': 'human', 'value': '<image>
🤌. Write ...
submission_id                                              1abgte5
Name: 9119, dtype: object

In [108]:
df.loc[9119]['conversations']

[{'from': 'human', 'value': '<image>\n🤌. Write an insult about this person.'},
 {'from': 'gpt',
  'value': "OP's Bio:  ---  >Dia dhaoibh (Hello Everyone)  >I'm Twisted 29 From Ireland, My past times consists of Gaming, Watching Anime, Studying Cyber security and outside of those I like wild water swimming, hiking, the outdoors, space and some Mary Jane.  > >Favourite Video Game - Skyrim >Favourite Book - The Hobbit  >Favourite Movies Top 3 >- Lotr Trilogy >- The Moulin Rouge  >- Dead Poet Society > >Music ( everything ) >Seriously I could go from Beethoven to I Prevail to Tiësto to Conway Twitty real quick. > >Hot Take >Feminism has been hijacked by woke, brainwashed Pawns and has ruined the Western World and Womens Rights and Morals.  --- If you think this bio helped you roast, upvote this comment. If you think it doesn’t, downvote it. If you’re not sure, leave it to others to decide."}]

In [112]:
df.loc[9119]['conversations'][1]['value']

"OP's Bio:  ---  >Dia dhaoibh (Hello Everyone)  >I'm Twisted 29 From Ireland, My past times consists of Gaming, Watching Anime, Studying Cyber security and outside of those I like wild water swimming, hiking, the outdoors, space and some Mary Jane.  > >Favourite Video Game - Skyrim >Favourite Book - The Hobbit  >Favourite Movies Top 3 >- Lotr Trilogy >- The Moulin Rouge  >- Dead Poet Society > >Music ( everything ) >Seriously I could go from Beethoven to I Prevail to Tiësto to Conway Twitty real quick. > >Hot Take >Feminism has been hijacked by woke, brainwashed Pawns and has ruined the Western World and Womens Rights and Morals.  --- If you think this bio helped you roast, upvote this comment. If you think it doesn’t, downvote it. If you’re not sure, leave it to others to decide."

In [115]:
df['conversations'].apply(lambda x: x[1]['value'])

0                         You'll never outrun those roots.
1        I don’t think the voices in your head will be ...
2        OP's Bio:  ---  >i once hooked up with someone...
3        Eternally single? Don't worry I think I found ...
4        You look like the girls on the STD posters at ...
                               ...                        
19793                     Make dental hygiene great again.
19794    The first person in history to receive a full ...
19795    This is what happens when you ban abortions in...
19796    You strike me as a person who looks up and dow...
19797    After reading these comments he’s gonna stomp ...
Name: conversations, Length: 19798, dtype: object

In [142]:
mask = df['conversations'].apply(lambda x: len(x[1]['value']) > 3000)
df[mask]

,id,title,image,comment_id,conversations,submission_id
18396,lhitpa_gmylqmn,"Just 2,5 years between both pictures .. what h...",/home/devonperoutky/LLaVA/dataset/images/lhitp...,gmylqmn,"[{'from': 'human', 'value': '<image> Just 2,5 ...",lhitpa


In [131]:
df.loc[18396]['conversations']

[{'from': 'human',
  'value': '<image>\nJust 2,5 years between both pictures .. what happend to me .. roast me! Write an insult about this person.'},
 {'from': 'gpt',
  'value': 'You look like someone who has been carrying a heavy mental load on their back for their entire life. There’s stress and a heaviness in your eyes even in your first picture, OP.   Other commenters shared a link to another post you made, OP. As someone who has also struggled through childhood trauma, please know that therapy and trauma work based around CPTSD has been monumentally helpful for me.   If therapy isn’t an option right now, I can recommend some books and subreddits. It’s morbidly comforting knowing how many others can relate—to not feel alone with the struggles of trauma.   CPTSD: From Surviving to Thriving by Pete Walker (this book helped me put into context the lasting affects trauma had on me. I realized my brain was reacting in a way to protect itself and how it’s still using the same system to “

In [122]:
df.loc[7941]['conversations']

[{'from': 'human',
  'value': '<image>\n40 years old, wife walked out 2 years ago, stoner, 100k in credit debt, lives with parents in a retirement mobile home community, in love with a new girl who doesn’t like me back. On the bright side, I have a full time job, I travel the world and will be pursuing stand-up comedy in 2024. Destroy me! Write an insult about this person.'},
 {'from': 'gpt', 'value': '40?!? When?? In 1992?'}]